<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_textsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Text Summarization**
Refs:
- https://huggingface.co/models
- https://huggingface.co/models?search=bertabs
- [BertForMaskedLM](https://github.com/huggingface/transformers/blob/13aa174112f0c2ee794c44188ecf13b241694db0/src/transformers/modeling_bert.py#L876) used as architecture in BertAbs model

Hyper-params:
TBA

Issues:
1. Max sentence length: 64 (truncated)

## Environment Setup
To get the source code of Huggingface transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [22]:
!uname -a
!pip install wget
!apt-get install file

Linux df99a98cd6bd 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
file is already the newest version (1:5.32-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [0]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

In [24]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


## Configuratiuon

In [0]:
EXAMPLE_MODE=True
INLINE_MODE=False

In [26]:
if EXAMPLE_MODE:
  # ! [ ! -d "./transformers" ] && git clone --depth 1 https://github.com/huggingface/transformers
  # ! [ -d "./transformers" ] && \
  #   cd transformers && \
  #   git fetch -a --tags && \
  #   git checkout tags/v2.11.0 && \
  #   pip install . && \
  !rm -rf ./nlp-qa/
  ![ ! -d "./nlp-qa" ] && git clone --depth 1 https://github.com/hailusong/nlp-qa.git
  !pip install nltk py-rouge

Cloning into 'nlp-qa'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 44 (delta 2), reused 28 (delta 2), pack-reused 0
Unpacking objects: 100% (44/44), done.


### GPU Setup

In [27]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


## Download

### Pre-trained BERTABS fine-tuned on CNN/DM corpus 


In [0]:
if INLINE_MODE:
  # from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  # from transformers import AlbertConfig, AlbertForSequenceClassification
  # import torch
  from modeling_bertabs import BertAbs, build_predictor
  from transformers import BertTokenizer

  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
  model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
  model.to(args.device)
  model.eval()

### Text to be summarized

In [29]:
if EXAMPLE_MODE:
  %env DATA_PATH=/content/nlp-qa/stories
  %env SUMMARIES_PATH=/content/textsum-out

  !mkdir -p $SUMMARIES_PATH
  !ls -al $SUMMARIES_PATH


env: DATA_PATH=/content/nlp-qa/stories
env: SUMMARIES_PATH=/content/textsum-out
total 8
drwxr-xr-x 2 root root 4096 Jun  9 22:09 .
drwxr-xr-x 1 root root 4096 Jun  9 22:12 ..


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [38]:
if EXAMPLE_MODE:
  # !pwd
  # !ls -al transformers/examples/summarization/bertabs
  # !cd transformers/examples/summarization/bertabs && python run_summarization.py \
  #   --documents_dir $DATA_PATH \
  #   --summaries_output_dir $SUMMARIES_PATH \
  #   --no_cuda false \
  #   --batch_size 4 \
  #   --min_length 50 \
  #   --max_length 200 \
  #   --beam_size 5 \
  #   --alpha 0.95 \
  #   --block_trigram true
  # !ls -al nlp-qa/textsum/bertabs
  !cd nlp-qa/textsum/bertabs && python run_summarization.py \
    --documents_dir $DATA_PATH \
    --summaries_output_dir $SUMMARIES_PATH \
    --no_cuda false \
    --batch_size 4 \
    --min_length 20 \
    --max_length 50 \
    --beam_size 5 \
    --alpha 0.95 \
    --block_trigram true

2020-06-09 22:50:19.306087: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/remi/bertabs-finetuned-cnndm-extractive-abstractive-summarization/config.json from cache at /root/.cache/torch/transformers/9d982039993bba60a064b70f3d5733f091aefc83eef2f1cd222242ca2f4fd0d9.d524a81afda9167a3e76519785c0610c76f5647e45b9c17a8849f7fe0088cf5f
INFO:transformers.configuration_utils:Model config BertAbsConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "dec_dropout": 0.2,
 

In [39]:
!ls -al $SUMMARIES_PATH
!grep "" $SUMMARIES_PATH/*

total 36
drwxr-xr-x 2 root root 4096 Jun  9 22:14 .
drwxr-xr-x 1 root root 4096 Jun  9 22:12 ..
-rw-r--r-- 1 root root  206 Jun  9 22:50 e117408ad19cc69e15b1e21b9ae54f10c07223ce_summary.story
-rw-r--r-- 1 root root  252 Jun  9 22:50 e3dd06d326c8d53722bdc5c8428e29c68a60d0d5_summary.story
-rw-r--r-- 1 root root  228 Jun  9 22:50 e9093ca0a82f2aba28cd2762942c04177cfbb000_summary.story
-rw-r--r-- 1 root root  198 Jun  9 22:51 ea06fd0b25cb9793397a51de73fd83f91b4323fa_summary.story
-rw-r--r-- 1 root root  243 Jun  9 22:51 eb68bc51ed4fc727a1af058192a3fef0916c91e7_summary.story
-rw-r--r-- 1 root root  267 Jun  9 22:50 ee0ba7928d8987f2cb21c9a2012a76730f77de45_summary.story
-rw-r--r-- 1 root root  218 Jun  9 22:51 ee8871b15c50d0db17b0179a6d2beab35065f1e9_summary.story
/content/textsum-out/e117408ad19cc69e15b1e21b9ae54f10c07223ce_summary.story:the partnership is an organization with partners throughout the uk. it employs approximately 67 , 100 people. each partner receives the same scale of bonus 